In [3]:
import pandas as pd 
import numpy as np 
import xmltodict
import base64
import struct
import argparse
import os
import sys

import torch.nn as nn
from torch.utils.data import TensorDataset
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
# import torchvision
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
# from torchvision import datasets, models, transforms
# from torchvision.transforms import Resize, ToTensor, Normalize
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
import cv2
from scipy import stats
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split
import time
import os
from pathlib import Path
from skimage import io
import copy
from torch import optim, cuda
import glob
from collections import Counter
# Useful for examining network
from functools import reduce
from operator import __add__
# from torchsummary import summary
import seaborn as sns
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from PIL import Image
from timeit import default_timer as timer
import matplotlib.pyplot as plt

# Useful for examining network
from functools import reduce
from operator import __add__
# from torchsummary import summary

# from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns

import warnings
# warnings.filterwarnings('ignore', category=FutureWarning)

# Image manipulations
from PIL import Image

# Timing utility
from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt

# Baseline wander removal
import scipy
import scipy.signal

# Convert XML files into .npy arrays (user changes XML_input_directory_path)

In [6]:
home_dir = os.getcwd()

In [11]:
XML_input_directory_path = '/insert/path/to/XML/files' #this should be changed
# XML_input_directory_path = '/Users/pae2/Box/Data/muse_XML_waveform_miniset/' #this should be changed
npy_output_directory_path = os.path.join(home_dir,'ekg_waveform_output') #this can be left as default
query_metadata_result = '/path/to/query_metadata_result.csv' #this should be changed 
# query_metadata_result = '/Users/pae2/Box/Data/MuseLabelGeneration/Master_File_MUSE_Pt_List.csv'
fpath_eval_df_newest_ecg_waveform_data_prebuilt = 'path/to/prebuilt_array.npy' #stacked waveform data of newest ecg per pt, will build below as [X,2500,12,1]
fpath_eval_df_newest_ecg_waveform_data = os.path.join(home_dir,'eval_newest_ecg_per_pt_waveform_features.npy') #this can be left as default
fpath_eval_df_newest_ecg_waveform_data_wander_removed = os.path.join(home_dir,'eval_newest_ecg_per_pt_wander_removed_waveform_features.npy')
fpath_eval_df_newest_ecg_waveform_data_wander_removed_pct_truncated_mean_normalized = os.path.join(home_dir,'eval_newest_ecg_per_pt_wander_removed_pct_truncated_mean_normalized_waveform_features.npy')
#The following would be for  the binary labels you should provide
newest_ecg_per_pt_eval_labels = os.path.join(home_dir,'eval_newest_ecg_per_pt_wander_removed_pct_truncated_mean_normalized_label.npy')


In [13]:
def file_path(path):
    filepath = path
    for dirName, subdirList, fileList in os.walk(filepath):
        for filename in fileList:
            if ".xml" in filename.lower():
                ekg_file_list.append(os.path.join(dirName,filename))

dir_name = npy_output_directory_path+'/'

if not os.path.exists(dir_name):
    os.mkdir(dir_name)

def decode_ekg_muse(raw_wave):
    """
    Ingest the base64 encoded waveforms and transform to numeric
    """
    # covert the waveform from base64 to byte array
    arr = base64.b64decode(bytes(raw_wave, 'utf-8'))

    # unpack every 2 bytes, little endian (16 bit encoding)
    unpack_symbols = ''.join([char*int(len(arr)/2) for char in 'h'])
    byte_array = struct.unpack(unpack_symbols,  arr)
    return byte_array


def decode_ekg_muse_to_array(raw_wave, downsample = 1):
    """
    Ingest the base64 encoded waveforms and transform to numeric

    downsample: 0.5 takes every other value in the array. Muse samples at 500/s and the sample model requires 250/s. So take every other.
    """
    try:
        dwnsmpl = int(1//downsample)
    except ZeroDivisionError:
        print("You must downsample by more than 0")
    # covert the waveform from base64 to byte array
    arr = base64.b64decode(bytes(raw_wave, 'utf-8'))

    # unpack every 2 bytes, little endian (16 bit encoding)
    unpack_symbols = ''.join([char*int(len(arr)/2) for char in 'h'])
    byte_array = struct.unpack(unpack_symbols,  arr)
    return np.array(byte_array)[::dwnsmpl]


def xml_to_np_array_file(path_to_xml, path_to_output = home_dir):

    with open(path_to_xml, 'rb') as fd:
        dic = xmltodict.parse(fd.read().decode('utf8'))

    """

    Upload the ECG as numpy array with shape=[2500,12,1] ([time, leads, 1]).

    The voltage unit should be in 1 mv/unit and the sampling rate should be 250/second (total 10 second).

    The leads should be ordered as follow I, II, III, aVR, aVL, aVF, V1, V2, V3, V4, V5, V6.

    """
    try:
        pt_id = dic['RestingECG']['PatientDemographics']['PatientID']
    except:
        print("no PatientID")
        pt_id = "none"
    try:
        PharmaUniqueECGID = dic['RestingECG']['PharmaData']['PharmaUniqueECGID']
    except:
        print("no PharmaUniqueECGID")
        PharmaUniqueECGID = "none"
    try:
        AcquisitionDateTime = dic['RestingECG']['TestDemographics']['AcquisitionDate'] + "_" + dic['RestingECG']['TestDemographics']['AcquisitionTime'].replace(":","-")
    except:
        print("no AcquisitionDateTime")
        AcquisitionDateTime = "none"    
        
    #need to instantiate leads in the proper order for the model
    lead_order = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']

    """
    Each EKG will have this data structure:
    lead_data = {
        'I': np.array
    }
    """

    lead_data =  dict.fromkeys(lead_order)

    for lead in dic['RestingECG']['Waveform']:
        for leadid in range(len(lead['LeadData'])):
                sample_length = len(decode_ekg_muse_to_array(lead['LeadData'][leadid]['WaveFormData']))
                #sample_length is equivalent to dic['RestingECG']['Waveform']['LeadData']['LeadSampleCountTotal']
                if sample_length == 5000:
                    lead_data[lead['LeadData'][leadid]['LeadID']] = decode_ekg_muse_to_array(lead['LeadData'][leadid]['WaveFormData'], downsample = 0.5)
                elif sample_length == 2500:
                    lead_data[lead['LeadData'][leadid]['LeadID']] = decode_ekg_muse_to_array(lead['LeadData'][leadid]['WaveFormData'], downsample = 1)
                else:
                    continue
            #ensures all leads have 2500 samples and also passes over the 3 second waveform

    lead_data['III'] = (np.array(lead_data["II"]) - np.array(lead_data["I"]))
    lead_data['aVR'] = -(np.array(lead_data["I"]) + np.array(lead_data["II"]))/2
    lead_data['aVF'] = (np.array(lead_data["II"]) + np.array(lead_data["III"]))/2
    lead_data['aVL'] = (np.array(lead_data["I"]) - np.array(lead_data["III"]))/2
    
    lead_data = {k: lead_data[k] for k in lead_order}
    # drops V3R, V4R, and V7 if it was a 15-lead ECG

    # now construct and reshape the array
    # converting the dictionary to an npy.array
    temp = []
    for key,value in lead_data.items():
        temp.append(value)

    #transpose to be [time, leads, ]
    ekg_array = np.array(temp).T

    #expand dims to [time, leads, 1]
    ekg_array = np.expand_dims(ekg_array, axis=-1)

    # Here is a check to make sure all the model inputs are the right shape
#     assert ekg_array.shape == (2500, 12, 1), "ekg_array is shape {} not (2500, 12, 1)".format(ekg_array.shape )

    filename = '{}_{}_{}.npy'.format(pt_id, AcquisitionDateTime,PharmaUniqueECGID)

    path_to_output += '/'+filename
    # print(path_to_output)
    if not os.path.exists(dir_name + '/' + filename):
        with open(path_to_output, 'wb') as f:
            np.save(f, ekg_array)
        

def ekg_batch_run(ekg_list):
    i = 0
    x = 0
    for file in ekg_list:
        try:
            xml_to_np_array_file(file, output_dir)
            i+=1
        except Exception as e:
            # print("file failed: ", file)
            print(file, e)
            x+=1
        if i % 10000 == 0:
            print(f"Succesfully converted {i} EKGs, failed converting {x} EKGs")

dir_name = npy_output_directory_path
output_dir = dir_name
ekg_file_list = []
file_path(XML_input_directory_path)  #if you want input to be a directory
print("Number of EKGs found: ", len(ekg_file_list))
#print(type(ekg_file_list))
ekg_file_list.sort(reverse = True)
print("The newest EKG is: ", ekg_file_list[0])
ekg_batch_run(ekg_file_list)

# To reconstruct the 12 lead ecg from the array
# test1 = np.load('waveform_output_example.npy')
# lead_order = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
# plt.rcParams["figure.figsize"] = [16,9]
# fig, axs = plt.subplots(len(lead_data))
# for i in range(0,12):
#     axs[i].plot(test1[:,i])
#     axs[i].set(ylabel=str(lead_order[i]))

Number of EKGs found:  3342
The newest EKG is:  /Users/pae2/Box/Data/muse_XML_waveform_miniset/xmls/MUSE_XXXXXXXXXX.xml
no PharmaUniqueECGID
no PharmaUniqueECGID


In [ ]:
'''
SELECT tdem.TestID, 
  max(pt.PatientID) as PatientID, 
  max(pt.DateofBirth_D) as DateofBirth_D, 
  max(Gender) as Gender, 
  max(Race) as Race,
  max(AcquisitionDateTime_DT) as AcquisitionDateTime_DT,
  max(CASE when AgeUnits = 0 and PatientAge > 0 then PatientAge ELSE null END) as PatientAge_Years,
  max(ecg.UniqueECGID) as UniqueECGID,
  (CAST(CASE WHEN tdem.TestID in 
        (
        SELECT det.TestID
        FROM BIS.MUSE_Site0013.tstDiagnosisDetails det (nolock) 
        LEFT JOIN BIS.MUSE_Site0013.cfgStatements st (nolock)on st.StatementNumber = det.StatementNumber  
        WHERE 
        Acronym in ('VPCX', 'AVPCX', 'ASVPR', 'ASVPC', 'AVPCK', 'PCK', 'BIVPCK', 'VPR', 'AVDPR', 'WITH-DEM')
        )
        THEN 1 
    ELSE 0
    END AS int)) as Ventricular_Pacing_Present,
  (CAST(CASE WHEN tdem.TestID in 
        (
        SELECT det.TestID
        FROM BIS.MUSE_Site0013.tstDiagnosisDetails det (nolock) 
        LEFT JOIN BIS.MUSE_Site0013.cfgStatements st (nolock)on st.StatementNumber = det.StatementNumber  
        WHERE 
        Acronym in ('QCERR')
        )
        THEN 1 
    ELSE 0
    END AS int)) as Poor_Data_Quality_Present,
    max(tst.AtrialRate) as AtrialRate,
    max(tst.VentricularRate) as VentricularRate,
    max(tst.P_RInterval) as P_RInterval,
    max(tst.QRSDuration) as QRSDuration,
	max(tst.QTCCalculation) as QTCCalculation,
    max(tst.QTcFredericia) as QTcFredericia
into #muse_pt_list
  FROM BIS.MUSE_Site0013.tstTestDemographics tdem (nolocK) 
  LEFT JOIN BIS.MUSE_Site0013.tstDiagnosisDetails det (nolock) on tdem.TestID = det.TestID
  LEFT JOIN BIS.MUSE_Site0013.cfgTestTypeQualifiers ctq (nolock) on ctq.TestType = tdem.TestType
  LEFT JOIN BIS.MUSE_Site0013.cfgStatements st (nolock)on st.StatementNumber = det.StatementNumber  and st.TestType = tdem.TestType
  LEFT JOIN BIS.MUSE_Site0013.tstPatientDemographics pt (nolock) on pt.TestID = tdem.TestID
  LEFT JOIN BIS.MUSE_Site0013.tstPharmaTestInformation ecg (nolock) on ecg.TestID = tdem.TestID
  LEFT JOIN BIS.MUSE_Site0013.tstRestingECGMeasurement tst (nolock) on tst.TestID = tdem.TestID
  WHERE 
  tdem.TestID > 0
  and PatientID <> 'NO PID'
  GROUP BY tdem.TESTID
  ORDER BY TESTID desc
  '''

#export these results as a csv file with the path/filename.csv that you put in query_metadata_result above

## Alternatively if you do not use SQL queries to access metadata and instead are comfortable with extracting diagnostic statements from the XML file, the associated full text diagnostic statements are below. Caution, I have not used this approach in the past so there may be site differences that I am unaware of. 

In [4]:
full_text_statements = ['*** POOR DATA QUALITY, INTERPRETATION MAY BE ADVERSELY AFFECTED',
'ATRIAL-SENSED VENTRICULAR-PACED COMPLEXES',
'ATRIAL-SENSED VENTRICULAR-PACED RHYTHM',
'AV DUAL-PACED COMPLEXES',
'AV DUAL-PACED RHYTHM',
'AV SEQUENTIAL OR DUAL CHAMBER ELECTRONIC PACEMAKER',
'BIVENTRICULAR PACEMAKER DETECTED',
'ELECTRONIC VENTRICULAR PACEMAKER',
'VENTRICULAR- PACED COMPLEXES',
'VENTRICULAR-PACED RHYTHM',
'WITH A DEMAND PACEMAKER']

## From here we generate a csv file (the path/filename you put into query_metadata_result) with the metadata from the above query, including: 
- ['TestID', 'PatientID', 'DateofBirth_D', 'Gender', 'AcquisitionDateTime_DT', 'PatientAge_Years', 'UniqueECGID', 'Ventricular_Pacing_Present', 'Poor_Data_Quality_Present', 'AtrialRate', 'VentricularRate', 'P_RInterval', 'QRSDuration', 'QTCCalculation', 'QTcFredericia']<br>

## That is exported as a csv. The function below generates a filename that will match the files you created above and exported into /ekg_waveform_output

In [14]:
eval_df = pd.read_csv(query_metadata_result)

def add_filename(df): 
    df['AcquisitionDateTime_DT'] = pd.to_datetime(df['AcquisitionDateTime_DT'], errors='coerce')
    df['FileFormatDT'] = df['AcquisitionDateTime_DT'].dt.strftime('%m-%d-%Y_%H-%M-%S')
    FileFormatDT = ['none' if x == 'NaT' else x for x in df['FileFormatDT']] #note that the NaT here is not pd.NaT but just string 
    FileFormatDT.count('none')
    UniqueECGID = ['none' if pd.isnull(x)==True else x for x in df['UniqueECGID']] 
    UniqueECGID.count('none')
    df['PatientID'] = df['PatientID'].astype(str).replace('\.0', '', regex=True)
    PatientID = ['none' if x == 'nan' else x for x in df['PatientID']]
    filename = []

    for i in range(len(PatientID)):
        name = str(PatientID[i]) +'_'+ str(FileFormatDT[i]) +'_'+ str(UniqueECGID[i]) + '.npy'
        filename.append(name)
    df['filename'] = filename
    
if 'filename' not in eval_df.columns:
    add_filename(eval_df)

/Users/pae2/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Remove poor data quality and ventricularly paced ECGs

In [15]:
eval_df = eval_df[eval_df['Ventricular_Pacing_Present']==0]
eval_df = eval_df[eval_df['Poor_Data_Quality_Present']==0]

# Add age, fill in missing values, and standard scale tabular data columns 

In [16]:
#only if Age not already calculated, uncomment below
def generate_age_multisite(df,study_date,dob):
    df[study_date] = pd.to_datetime(df[study_date],errors='coerce')
    df[dob] = pd.to_datetime(df[dob],errors='coerce')
    DateofBirth_D = []
    df['PatientAge_Years'] = (df[study_date] - df[dob]).dt.days//365
    print(f'Number of rows where age not calculated:{np.sum(df.Age.isnull())}')
    return df
# eval_df = generate_age_multisite(eval_df,'AcquisitionDateTime_DT','DateofBirth_D')

In [17]:
def binarize(df,column):
    binary = np.array(df[column]).reshape(-1, 1)
    enc = preprocessing.MinMaxScaler()
    binary_scaled = enc.fit_transform(binary)
    df[column] = binary_scaled
    
def binarize_standard(df,column):
    binary = np.array(df[column]).reshape(-1, 1)
    enc = preprocessing.StandardScaler()
    binary_scaled = enc.fit_transform(binary)
    df[column] = binary_scaled
    
enc = preprocessing.OneHotEncoder(sparse=False)

In [28]:
#drop any duplicate rows that occur due to poor naming 
eval_df = eval_df.drop_duplicates(subset='filename',keep='last')

# <ins>Ensure waveforms exist for the metadata</ins>
### Once you have restricted to ECGs meeting above criteria, select only the newest ECG per patient 

In [29]:
eval_df_available_path = Path("none.csv")
print("make eval_df_available", time.process_time()/100000,'seconds')
if not Path(eval_df_available_path).exists():
    print("No eval_df_available found, creating now...")
    npyfilespath = npy_output_directory_path
    os.chdir(npyfilespath)
    npfiles = glob.glob("*.npy")
    npfiles.sort()
    npdf = pd.DataFrame({'filename': npfiles})
    
    eval_df_available = eval_df.merge(npdf,left_on='filename', right_on='filename', validate='1:1', suffixes=('_df',''))
    eval_df_available.to_csv("./eval_df_metadata_available.csv",index=False)
else:
    print('eval_df_available found, passing in eval_df')
    eval_df_available = eval_df

filelist_eval_df = eval_df_available['filename']

print('eval_df length:', len(eval_df))
print('eval_df available length:', len(filelist_eval_df))
print('eval_df ECG count:', len(eval_df_available))

make eval_df_available 0.0008439438800000001 seconds
No eval_df_available found, creating now...
eval_df length: 3192328
eval_df available length: 2917
eval_df ECG count: 2917


In [31]:
eval_df_available['AcquisitionDateTime_DT'] = pd.to_datetime(eval_df_available['AcquisitionDateTime_DT'], errors='coerce')
eval_newest_ecg_per_pt = eval_df_available.loc[eval_df_available.groupby("PatientID")["AcquisitionDateTime_DT"].idxmax()]
assert eval_newest_ecg_per_pt['PatientID'].nunique() == len(eval_newest_ecg_per_pt), 'There is not one ECG per PatientID'

In [32]:
eval_newest_ecg_per_pt_tabular = eval_newest_ecg_per_pt[TABULAR_COLUMNS_TRUNCATED]
assert np.isnan(eval_newest_ecg_per_pt_tabular).sum(axis=0).sum() == 0, "Error! eval_newest_ecg_per_pt_tabular has NaNs"

In [34]:
np.save("eval_newest_ecg_per_pt_tabular.npy", np.array(eval_newest_ecg_per_pt_tabular).astype('float64'))
eval_newest_ecg_per_pt.to_csv('eval_newest_ecg_per_pt_tabular_metadata.csv',index=False)

# <ins>Save Labels</ins>

### These should be saved as npy arrays with binary coding under the paths you add above
newest_ecg_per_pt_eval_labels = 'eval_newest_ecg_per_pt_wander_removed_pct_truncated_mean_normalized_label.npy'<br>


# <ins>Build Stacked Waveform Array</ins>

In [43]:
try:
    eval_df_newest_ecg_waveform_data = np.load(fpath_eval_df_newest_ecg_waveform_data_prebuilt)  # Load input data. Input data should be compiled as numpy arrays with (sample numbers, time, lead, 1)
    print('Successfully loaded pre-built eval_df_newest_ecg_waveform_data')
except:
    eval_df_newest_ecg_waveform_data = []

time1 = timer()
if not Path(fpath_eval_df_newest_ecg_waveform_data_prebuilt).exists():
    print("no eval_df array found, building now...")
    for npfile in filelist_eval_df:
        i = 0
        try:
            path = os.path.join(npyfilespath, npfile)
            file = np.load(path)
            eval_df_newest_ecg_waveform_data.append(file)
            i += 1
            if i % 1 == 100:
                print("{i} EKGs have been written to array")
        except:
            continue
    eval_df_newest_ecg_waveform_data = np.array(eval_df_newest_ecg_waveform_data)
    np.save(fpath_eval_df_newest_ecg_waveform_data, eval_df_newest_ecg_waveform_data)

print("time to generate arrays: ", timer() - time1)
print(f'eval_df_newest_ecg_waveform_data shape: {eval_df_newest_ecg_waveform_data.shape}')

no eval_df array found, building now...
time to generate arrays:  5.379244727000014
eval_df_newest_ecg_waveform_data shape: (2917, 2500, 12, 1)


# <ins>Baseline Wander Removal</ins>
*Note that this process is slow and only does around 240 ecgs per minute. I recommend running this notebook in screen so that this can run over the course of many hours. If you have a compute cluster and can multi-thread this it can run much faster but we haven't built that yet.*

In [44]:
baseline_wander_removed_eval_df_newest_ecg_waveform_data = []

def baseline_wander_removal(data, sampling_frequency):
    row,__ = data.shape
    processed_data = np.zeros(data.shape)
    for lead in range(0,row):
        # Baseline estimation
        win_size = int(np.round(0.2 * sampling_frequency)) + 1
        baseline = scipy.signal.medfilt(data[lead,:], win_size)
        win_size = int(np.round(0.6 * sampling_frequency)) + 1
        baseline = scipy.signal.medfilt(baseline, win_size)
        # Removing baseline
        filt_data = data[lead,:] - baseline
        processed_data[lead,:] = filt_data
    return processed_data

def baseline_wander_removal_batch(dataset,outputlist):
    a=0
    time1 = time.time()

    if dataset.shape[1:] == (2500,12,1):
        print('dataset shape:',dataset.shape,'transposing now...')
        dataset = np.transpose(dataset, axes=[0,2,1,3])
    elif dataset.shape[1:] == (1,2500,12):
        print('dataset shape:', dataset.shape, 'transposing now...')
        dataset = np.transpose(dataset, axes=[0,3,2,1])
    elif dataset.shape[1:] == (12,2500,1):
        pass
    for ecg in dataset:
        assert ecg.shape == (12,2500,1), "ecg is not 12,2500"
        a+=1
        processed_data = baseline_wander_removal(ecg.squeeze(),250)
        outputlist.append(processed_data)
        if a%1000==0:
            print(a,'ECGs converted')
            print(time.time()-time1,"seconds since start")

    outputlist = np.expand_dims(np.array(outputlist),axis=3)
    print(outputlist.shape)

baseline_wander_removal_batch(eval_df_newest_ecg_waveform_data,baseline_wander_removed_eval_df_newest_ecg_waveform_data)

np.save(fpath_eval_df_newest_ecg_waveform_data_wander_removed,baseline_wander_removed_eval_df_newest_ecg_waveform_data)
print('completed wander removal')

dataset shape: (2917, 2500, 12, 1) transposing now...
1000 ECGs converted
225.57666420936584 seconds since start
2000 ECGs converted
456.98662400245667 seconds since start
(2917, 12, 2500, 1)
completed wander removal


# <ins>Per-Lead Truncation and Mean Normalization</ins>
*We've now dealt with a major form of bias by removing baseline wander. But there are still significant outliers that are physiologically unimportant. So we truncate to the 0.1st and 99.9th percentile of the data internally, then do per-lead normalization using the mean and STD of the Columbia train dataset*

In [47]:
def truncate_data_percentile(target_array, lowerbound, upperbound):
    datapctlimit = target_array
    assert (len(upperbound) == datapctlimit.shape[1]), "shape of upperbound does not match array.shape[1]"
    # Truncate data to 0.1th and 99.9th percentile
    for i in range(len(upperbound)):
        print("pre-truncation min:",datapctlimit[:, i, :, :].min())
        print("pre-truncation max:", datapctlimit[:, i, :, :].max())
        datapctlimit[:, i, :, :] = np.where(datapctlimit[:, i, :, :] > upperbound[i], upperbound[i], datapctlimit[:, i, :, :])
        datapctlimit[:, i, :, :] = np.where(datapctlimit[:, i, :, :] < lowerbound[i], lowerbound[i], datapctlimit[:, i, :, :])
    for i in range(len(upperbound)):
        print("post-truncation min:",datapctlimit[:, i, :, :].min())
        print("post-truncation max:", datapctlimit[:, i, :, :].max())
    # datanorm2 = cv2.normalize(datapctlimit, datapctlimit, -1, 1, cv2.NORM_MINMAX)
    return datapctlimit

def per_lead_truncation_normalization_prospective(dataset_data,save_path):
    if dataset_data.shape[1:] == (2500,12,1):
        print('dataset_data shape:',dataset_data.shape,'transposing now...')
        dataset_data = np.transpose(dataset_data, axes=[0,2,1,3])
    elif dataset_data.shape[1:] == (1,2500,12):
        print('dataset_data shape:', dataset_data.shape, 'transposing now...')
        dataset_data = np.transpose(dataset_data, axes=[0,3,2,1])
    elif dataset_data.shape[1:] == (12,2500,1):
        pass
    assert dataset_data.shape[1:] == (12, 2500, 1), "dataset is not X,12,2500,1"

    # Compute mean and standard deviation for normalization
    mean_tr, std_tr = np.mean(dataset_data, axis=(0, 2, 3)), np.std(dataset_data, axis=(0, 2, 3))
    print('Mean and STD before truncation')
    print(mean_tr)
    print(std_tr)

    # med_tr, mad_tr = np.median(dataset_data, axis=(0, 2, 3)), stats.median_absolute_deviation(dataset_data, axis=(0, 2, 3))
    # print("Median and MAD before truncation", mean_tr, std_tr)

    lowerbound = []
    upperbound = []
    for i in range(0,12):
        dataset = dataset_data
        lowerbound.append(np.percentile(dataset[:,i,:,:],0.1))
        upperbound.append(np.percentile(dataset[:,i,:,:],99.9))

    print("dataset 0.1st percentile lowerbound:",lowerbound)
    print("dataset 99.9th percentile upperbound:",upperbound)


    # NOTE: NEED TO put Muse_Union_Syngo_data before dataset_data 
    dataset_data = truncate_data_percentile(dataset_data, lowerbound, upperbound)
    print("dataset data truncated based on percentiles")

    mean_tr, std_tr = np.mean(dataset_data, axis=(0, 2, 3)), np.std(dataset_data, axis=(0, 2, 3))
    print('Mean and STD before normalization', mean_tr, std_tr)

    # For validation and prospective trial must use train mean and standard deviation
    # These values comes from /labs/amyloid/CardiacAmyloid/ValveNetExtraData/220k_multivalve_train_pace_removed_poor_quality_removed_wander_removed_waveform_features.npy
    mean_tr = [5.3913547,4.08127771,-1.18338815,-4.72457889,0.59169407,-0.59169407,-4.25676191,-1.44444217,-0.34211766,3.35517074,5.18700818,5.53535442]
    std_tr = [31.1319959,29.16014537,30.41666735,26.0615433,15.20833367,15.20833367,38.35645889,53.62205647,58.92661479,49.45070532,43.92990916,38.29998916]

    # Per-lead normalization (there are 12 leads)
    assert (len(mean_tr) == dataset_data.shape[1])
    for i in range(len(mean_tr)):
        tic = time.perf_counter()
        print("pre-normalizing min:",dataset_data[:, i, :, :].min())
        print("pre-normalizing max:", dataset_data[:, i, :, :].max())
        dataset_data[:, i, :, :] = (dataset_data[:, i, :, :] - mean_tr[i]) / std_tr[i]
        print("post-normalizing min:",dataset_data[:, i, :, :].min())
        print("post-normalizing max:", dataset_data[:, i, :, :].max())
        toc = time.perf_counter()
        print(f'Processed iter {i} in {toc - tic:0.2f} seconds')

    mean_norm, std_norm = np.mean(dataset_data, axis=(0, 2, 3)), np.std(dataset_data, axis=(0, 2, 3))
    print('Mean and STD after normalization', mean_norm, std_norm)
    # med_norm, mad_norm = np.median(dataset_data, axis=(0, 2, 3)), stats.median_absolute_deviation(dataset_data, axis=(0, 2, 3))
    # print("Median and MAD after normalization", med_norm, mad_norm)

    # Transpose data for model
    X_dataset = np.transpose(dataset_data, axes=[0, 3, 2, 1])
    print('X_dataset shape for model:', X_dataset.shape)
    assert X_dataset.shape[1:] == (1, 2500, 12), "dataset is not X,1,2500,12"

    # Write out feature arrays
    dataset_features_path = save_path #'/path/file.npy'
    np.save(dataset_features_path, X_dataset)
    print('Saved feature arrays')

In [68]:
eval_df_newest_ecg_waveform_data_wander_removed = np.load(fpath_eval_df_newest_ecg_waveform_data_wander_removed)
if len(eval_df_newest_ecg_waveform_data_wander_removed.shape) == 3:
    eval_df_newest_ecg_waveform_data_wander_removed = np.expand_dims(eval_df_newest_ecg_waveform_data_wander_removed,axis=3)

#run the normalization function and save the output waveform array which whill then be used for evaluation
per_lead_truncation_normalization_prospective(eval_df_newest_ecg_waveform_data_wander_removed, fpath_eval_df_newest_ecg_waveform_data_wander_removed_pct_truncated_mean_normalized)

Mean and STD before truncation
[ 5.97941282  5.87813493  0.09237943 -5.93905903 -0.04618972  0.04618972
 -4.66775358 -0.95858238  1.09394858  5.22302448  6.79488708  6.6911616 ]
[32.31957555 33.95522112 31.36092133 29.19812889 15.68046067 15.68046067
 40.18049874 53.62923659 61.15692121 55.13048428 49.13032946 41.64655158]
dataset 0.1st percentile lowerbound: [-151.0, -167.0, -282.0, -257.5, -132.5, -141.0, -360.0, -489.0, -563.0, -424.0, -285.0, -202.0]
dataset 99.9th percentile upperbound: [292.0, 324.0, 265.0, 115.0, 141.0, 132.5, 241.0, 326.0, 413.0, 464.0, 463.0, 403.0]
pre-truncation min: -648.0
pre-truncation max: 732.0
pre-truncation min: -504.0
pre-truncation max: 754.0
pre-truncation min: -940.0
pre-truncation max: 916.0
pre-truncation min: -728.0
pre-truncation max: 564.0
pre-truncation min: -458.0
pre-truncation max: 470.0
pre-truncation min: -470.0
pre-truncation max: 458.0
pre-truncation min: -910.0
pre-truncation max: 1424.0
pre-truncation min: -1094.0
pre-truncation max